# Fragrantica perfume review clasifier (LSTM with stopword removed)

In [1]:
import pandas as pd
import requests
from os import path
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
# Model Name
MODEL_NAME = 'lstm_with_stopword_removed_05'

# HyperParameters
PAD_LEN = 200
NUM_WORDS = 5000
EMBEDDING = 200
BATCH_SIZE = 256


In [3]:
data_path = '../dataset/dataset_210626_215600.csv'
data_exist = path.exists(data_path)

if not data_exist:
    url = 'https://kyuuuw-nlp-dataset.s3.ap-northeast-2.amazonaws.com/fragrantica/dataset_210626_215600.csv'
    r = requests.get(url, allow_redirects=True)
    open(data_path, 'w').write(r.content)

In [4]:
data = pd.read_csv(data_path)

X_data = data['stopwords_removed']
y_data = data['label']

print(len(X_data))
print(len(y_data))


##### 토큰화 및 인덱스 부여

74779
74779


In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_data)
sequences = tokenizer.texts_to_sequences(X_data)

print(X_data[0])
print(len(sequences[0]))

['got', 'sample', 'today', 'year', 'old', 'daughter', 'thought', 'smelling', 'sprayed', 'card', 'rotten', 'fish', 'nearly', 'threw', 'immediately', 'however', 'later', 'evening', 'decided', 'give', 'fair', 'trial', 'sprayed', 'crook', 'elbows', 'rotting', 'fish', 'smell', 'time', 'got', 'definite', 'bit', 'funk', 'almost', 'urine', 'scent', 'minutes', 'blossomed', 'gorgeous', 'smooth', 'woody', 'ambery', 'clean', 'warm', 'jasmine', 'daughter', 'didnt', 'even', 'believe', 'told', 'fragrance', 'smelled', 'earlier', 'lol', 'immediately', 'commented', 'fresh', 'clean', 'vibe', 'agree', 'soapy', 'hint', 'powdery', 'way', 'obsessed', 'cant', 'stop', 'smelling', 'arm', 'glad', 'gave', 'try', 'wait', 'buy', 'full', 'bottle']
76


##### 인덱스별 단어 표시

In [6]:
word_to_index = tokenizer.word_index


##### 빈도수 분석

In [7]:
threshold = 2
total_cnt = len(word_to_index) # 총 단어의 수
rare_cnt = 0 # 등장 빈도 수가 threshold 보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    if value < threshold:
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print(f"등장 빈도가 {threshold}번 이하인 희귀 단어의 수: {rare_cnt}")
print(f"단어 집합(vocabulary)에서 희귀 단어의 비율: {rare_cnt / total_cnt}" )
print(f"전체 등장 빈도에서 희귀 단어 등장 빈도 비율: {rare_freq / total_freq}")

등장 빈도가 2번 이하인 희귀 단어의 수: 44912
단어 집합(vocabulary)에서 희귀 단어의 비율: 0.552307635549762
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 0.013189565348140857


##### 등장 횟수 상위 50000개의 word 만 사용

In [8]:
tokenizer = Tokenizer(num_words=NUM_WORDS)
tokenizer.fit_on_texts(X_data)
sequences = tokenizer.texts_to_sequences(X_data)

print(sequences[:5])

[[41, 102, 175, 207, 142, 1698, 134, 83, 145, 1237, 2100, 4180, 874, 2645, 466, 105, 190, 474, 404, 111, 1313, 4301, 145, 3232, 4180, 5, 17, 41, 1604, 38, 77, 2819, 2, 184, 249, 355, 129, 1269, 120, 70, 155, 1698, 646, 27, 273, 587, 4, 45, 1272, 333, 466, 2065, 39, 120, 198, 330, 533, 276, 170, 36, 1699, 343, 421, 83, 575, 477, 324, 60, 532, 43, 185, 12], [15, 2041, 310, 3487, 642, 8, 1, 1432, 55, 1, 7, 806, 496, 161, 16, 8, 689, 3292, 57, 100, 67, 25, 161, 37, 801, 16, 168, 16, 343, 1263, 1095, 494, 7, 47, 90, 849, 12, 1388, 838, 404, 111, 60, 145, 3487, 273, 1762, 428, 215, 115, 1546, 57, 7, 155, 810, 35, 20, 350, 5, 5, 155, 1103, 70, 139, 442, 36, 20, 228, 572, 52, 11, 689, 459, 82, 1494, 73, 248, 632, 273, 67, 444, 138, 4, 405, 3487, 22, 8, 100, 19, 33, 244, 213, 101, 686, 2234, 16, 16, 252, 228, 1190, 1813, 22, 115, 46, 19, 1042, 12, 331, 6, 331, 187, 224, 621, 1686, 43, 12, 3753, 236, 17, 310, 268, 11, 1539], [7, 132, 58, 385, 133, 7, 132, 47, 26, 9, 454, 41, 102, 375, 1317, 510,

##### Split data

In [9]:
X_train, X_test, y_train, y_test = train_test_split(sequences, y_data)
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

56084
56084
18695
18695


In [10]:
print(X_train[0])
print(y_train[0])

[269, 72, 4, 14, 90, 811, 1192, 432, 280, 80, 9, 1012, 1222, 655, 860, 21, 18, 92, 10, 941, 47, 529, 18, 23, 867, 11, 200, 56, 23, 1041, 14, 278, 1, 655, 35, 242, 2, 628, 50, 36, 10, 974, 1, 38, 2133, 1139, 22, 23, 357, 19, 187, 787, 231, 342, 261, 342, 8, 131, 75, 278, 221, 70, 130, 1210, 183, 978, 197, 2, 30, 245, 1070, 6, 12, 963, 139, 1042]
2.0


In [11]:
# padding and trimming
X_train = pad_sequences(X_train, maxlen=PAD_LEN)
X_test = pad_sequences(X_test, maxlen=PAD_LEN)

In [12]:
# One hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(y_train[0])

[0. 0. 1. 0.]


In [13]:
model = Sequential()
model.add(Embedding(NUM_WORDS, EMBEDDING))
model.add(LSTM(EMBEDDING))
model.add(Dense(4, activation='softmax'))

model.summary()

[2021-06-26 19:28:31.534 tensorflow-2-3-gpu--ml-g4dn-xlarge-794be025f5602a375b1b7feb8a0a:2390 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-06-26 19:28:31.559 tensorflow-2-3-gpu--ml-g4dn-xlarge-794be025f5602a375b1b7feb8a0a:2390 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 200)         1000000   
_________________________________________________________________
lstm (LSTM)                  (None, 200)               320800    
_________________________________________________________________
dense (Dense)                (None, 4)                 804       
Total params: 1,321,604
Trainable params: 1,321,604
Non-trainable params: 0
_________________________________________________________________


In [14]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint(f'../model/{MODEL_NAME}.h5', monitor='val_acc', mode='max')

In [15]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [16]:
history = model.fit(X_train, y_train,
                    batch_size=BATCH_SIZE, epochs=30, callbacks=[es, mc],
                    validation_data=(X_test, y_test))

Epoch 1/30
220/220 [==============================] - 16s 73ms/step - loss: 0.9607 - acc: 0.6074 - val_loss: 0.7612 - val_acc: 0.7156
Epoch 2/30
220/220 [==============================] - 16s 72ms/step - loss: 0.6637 - acc: 0.7525 - val_loss: 0.6861 - val_acc: 0.7405
Epoch 3/30
220/220 [==============================] - 16s 72ms/step - loss: 0.5952 - acc: 0.7775 - val_loss: 0.6996 - val_acc: 0.7364
Epoch 4/30
220/220 [==============================] - 16s 72ms/step - loss: 0.5765 - acc: 0.7852 - val_loss: 0.7255 - val_acc: 0.7346
Epoch 5/30
220/220 [==============================] - 16s 73ms/step - loss: 0.5466 - acc: 0.7950 - val_loss: 0.7285 - val_acc: 0.7345
Epoch 6/30
220/220 [==============================] - 16s 73ms/step - loss: 0.5238 - acc: 0.8056 - val_loss: 0.7522 - val_acc: 0.7322
Epoch 00006: early stopping
